In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ResNet_cifar10_eq as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [3]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.001
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 2.89654
step 1, cross_entropy 2.58545
step 2, cross_entropy 2.81499
step 3, cross_entropy 2.29392
step 4, cross_entropy 2.36579
step 5, cross_entropy 2.3963
step 6, cross_entropy 2.43635
step 7, cross_entropy 2.27357
step 8, cross_entropy 2.33809
step 9, cross_entropy 2.26
step 10, cross_entropy 2.35649
step 11, cross_entropy 2.39187
step 12, cross_entropy 2.39171
step 13, cross_entropy 2.31693
step 14, cross_entropy 2.42864
step 15, cross_entropy 2.29071
step 16, cross_entropy 2.21512
step 17, cross_entropy 2.27613
step 18, cross_entropy 2.14493
step 19, cross_entropy 2.37741


In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.01
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.15883
test accuracy 0.188
step 399, cross_entropy 1.82735
test accuracy 0.294
step 599, cross_entropy 2.08493
test accuracy 0.301
step 799, cross_entropy 2.08292
test accuracy 0.307
step 999, cross_entropy 1.90867
test accuracy 0.364
step 1199, cross_entropy 2.04864
test accuracy 0.363
step 1399, cross_entropy 1.70091
test accuracy 0.383
step 1599, cross_entropy 1.48823
test accuracy 0.332
step 1799, cross_entropy 1.62914
test accuracy 0.405
step 1999, cross_entropy 1.86493
test accuracy 0.366
step 2199, cross_entropy 1.41576
test accuracy 0.393
step 2399, cross_entropy 2.07958
test accuracy 0.414
step 2599, cross_entropy 1.32511
test accuracy 0.376
step 2799, cross_entropy 1.58335
test accuracy 0.424
step 2999, cross_entropy 0.873164
test accuracy 0.445
step 3199, cross_entropy 1.41735
test accuracy 0.437
step 3399, cross_entropy 1.55709
test accuracy 0.453
step 3599, cross_entropy 1.53999
test accuracy 0.442
step 3799, cross_entropy 1.56321
test accuracy 0.4

In [5]:
rate = 0.002
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.46187
test accuracy 0.714
step 399, cross_entropy 0.686075
test accuracy 0.714
step 599, cross_entropy 0.578408
test accuracy 0.727
step 799, cross_entropy 0.872791
test accuracy 0.727
step 999, cross_entropy 0.571404
test accuracy 0.699
step 1199, cross_entropy 0.504638
test accuracy 0.717
step 1399, cross_entropy 0.543868
test accuracy 0.734
step 1599, cross_entropy 1.1901
test accuracy 0.733
step 1799, cross_entropy 0.654072
test accuracy 0.755
step 1999, cross_entropy 0.639356
test accuracy 0.74
step 2199, cross_entropy 0.662534
test accuracy 0.74
step 2399, cross_entropy 0.485004
test accuracy 0.729
step 2599, cross_entropy 0.510837
test accuracy 0.7
step 2799, cross_entropy 0.511922
test accuracy 0.727
step 2999, cross_entropy 0.428548
test accuracy 0.751
step 3199, cross_entropy 0.645039
test accuracy 0.722
step 3399, cross_entropy 0.783066
test accuracy 0.734
step 3599, cross_entropy 0.732687
test accuracy 0.731
step 3799, cross_entropy 0.819027
test a

In [6]:
rate = 0.0004
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.660438
test accuracy 0.74
step 399, cross_entropy 0.739329
test accuracy 0.739
step 599, cross_entropy 0.788909
test accuracy 0.76
step 799, cross_entropy 0.416521
test accuracy 0.74
step 999, cross_entropy 0.826308
test accuracy 0.758
step 1199, cross_entropy 0.79133
test accuracy 0.751
step 1399, cross_entropy 0.556296
test accuracy 0.737
step 1599, cross_entropy 0.458861
test accuracy 0.746
step 1799, cross_entropy 0.379735
test accuracy 0.762
step 1999, cross_entropy 0.859843
test accuracy 0.729
step 2199, cross_entropy 0.583911
test accuracy 0.754
step 2399, cross_entropy 0.30434
test accuracy 0.751
step 2599, cross_entropy 0.491966
test accuracy 0.771
step 2799, cross_entropy 0.706278
test accuracy 0.764
step 2999, cross_entropy 0.490912
test accuracy 0.737
step 3199, cross_entropy 0.819828
test accuracy 0.765
step 3399, cross_entropy 0.430504
test accuracy 0.739
step 3599, cross_entropy 0.823086
test accuracy 0.754
step 3799, cross_entropy 0.616028
test